In [16]:
import os
import numpy as np
#import matplotlib.pyplot as plt
#from matplotlib.patches import Rectangle,Polygon
##### Load and test with onnx
import onnx
import onnxruntime as ort
import numpy as np

observation = np.zeros((1, 2)).astype(np.float32)
def simulate_run(start_state, file, n=3000):
    ort_sess = ort.InferenceSession(file)
    state = start_state
    print(f"Starting with: {state}")
    rPos = [state[0]]
    rVel = [state[1]]
    actions = []
    for i in range(0,n):
        action = ort_sess.run(None, {'input.1': [[state[0][0],state[1][0]]]})[0][0]
        print(f"Action: {action}")
        #action=100
        actions.append(action)
        t=0.1
        pos_0 = state[0][0]
        vel_0 = state[1][0]
        vel = action*t + vel_0
        pos = action*t**2/2 + vel_0*t + pos_0
        state[0]=pos
        state[1]=vel
        print(f"Next: {pos}, {vel}")
        rPos.append(state[0])
        rVel.append(state[1])
        if state[0] <= 0 or state[0] > 100:
            if state[0] <= 1:
                print("CRASH")
            break
    return rPos, rVel, actions

In [19]:
simulate_run([[0.1858], [-8.915]],"../../test/networks/acc-2000000-64-64-64-64-retrain-100000-200000-0.9.onnx")

Starting with: [[0.1858], [-8.915]]
Action: [100.00101]
Next: [-0.20569499], [1.0851011]
CRASH


([[0.1858], array([-0.20569499], dtype=float32)],
 [[-8.915], array([1.0851011], dtype=float32)],
 [array([100.00101], dtype=float32)])